This part is a machine learning algorithm to recognise faces that figure in the training data (images)

In [1]:
import os   # provides functions to interact with the operating system
import numpy as np
from PIL import Image 
import cv2 as cv 
import pickle

1- Loading images that will be used as dataset 

In [2]:
Face_recognition_base= os.path.dirname(os.path.abspath('Face-Recognition.ipynb')) # define the path to our base
Face_recognition_images = os.path.join(Face_recognition_base,"Images")

2- Create the training dataset

In [8]:
labels=[]
x_train=[]
y_labels=[]
id=0
face_casc = cv.CascadeClassifier('cascades/haarcascade_frontalface_alt2.xml') # Algorithm of face detection (Haar Cascade)

for root, dirs, images in os.walk(Face_recognition_images):
    for image in images :
        if image.endswith('png') or image.endswith('jpg') :
            path= os.path.join(root, image)
            label = os.path.basename(os.path.dirname(path)) # get labels that are the names of the directories
            # label = os.path.basename(root) // same as the previous line
            # Note : we need to make sure that each directory represents a label and it is an attached name (no spaces in it)
     
# Creation of the dictionnary containing faces labels and their identifiers (labels should be numbers not str)       
            if not (label in labels) :
                labels.append(label)
            label_id = {labels[i]: list(range(len(labels)))[i] for i in range(len(labels))}
              
                                           
# we will convert our images into arrays: 
            pil_im=Image.open(path).convert('L') # convert into grayscale cuz we need it in gray scale
            
            # after training data, the accuracy of the model was bad, so will resize images to work with identical ones.
            res_img= pil_im.resize((500,500), Image.ANTIALIAS)
            
            
            
            np_im=np.array(res_img, dtype='uint8')  # uint8 variable format for pixels

            faces= face_casc.detectMultiScale(np_im, scaleFactor=1.5, minNeighbors=5) 
            # the classifier accepts only np arrays, cv2 manipulates images as arrays not like pillow, so we don't need to convert
            for(x,y,w,h) in faces : 
                x_train.append(np_im[y:y+h,x:x+h])
                y_labels.append(label_id[label])
                                 

C:\Users\21379\AppData\Local\Temp\ipykernel_12284\43559983.py:25: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  res_img= pil_im.resize((500,500), Image.ANTIALIAS)


In [9]:
# save the images with their labels as a dataset (convert labels to binary format)

with open('labels.pickle','wb') as f :    # 'wb' w write in the file and b byte, labels.pickle c l nom du file
    pickle.dump(label_id, f)

In [10]:
# Training of the recogniser

recog= cv.face.LBPHFaceRecognizer_create()
recog.train(x_train,np.array(y_labels))
recog.save('trainner.yml')   

3- Prediction  (can be in a completely different python file if we have the trainner.yml file and we know what recognizer we chose 'LBPH in our case')

In [11]:
recog.read('trainner.yml')   #once the recognizer is trained. we can use it to predict without train it 
                             #(model pres entaine) just we read the training file associated.

In [13]:
# Load Label names from pickle

from cv2 import LINE_AA


Labelstest={}
with open('labels.pickle','rb') as f :   
    orig_Labelstest = pickle.load(f)
    inv_label={ v:k for k,v in orig_Labelstest.items() }  # we need to invert names and ids cuz the prediction gives us
                                                          # number and we have to get names

# testing on video capture 

cap = cv.VideoCapture(0)   # 0 for laptop camera
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)   # This kind of cascade classifier works only on gray images
    faces= face_casc.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for(x,y,w,h) in faces : 
        
        id_, conf = recog.predict(gray[y:y+h, x:x+h])
        if conf>45 and conf<85:          # we need to check the signification of conf !
            cv.putText(frame, inv_label[id_], (x,y), cv.FONT_HERSHEY_SIMPLEX, int('1'), (255,0,0), int('2'), cv.LINE_AA)
            
        cv.imshow('frame', frame)
        
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

The reason why the recognizer doesn't give the correct results may be the size of images, we didn't use identical size images so that can cause a problem in the prediction by next cuz the training was not that good. So we changed the data size, we need to garantee that we have good data.

